In [2]:
!pip install langchain_huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 473.0/473.0 kB 27.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.79
    Uninstalling langchain-core-0.3.79:
      Successfully uninstalled langchain-core-0.3.79
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.7 which is incompatible.


In [6]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

In [7]:
from langchain_core.runnables import RunnableParallel

In [13]:
llm = HuggingFaceEndpoint(repo_id='google/gemma-2-2b-it', task='text-generation', huggingfacehub_api_token='your api key')

model = ChatHuggingFace(llm=llm)

prompt1 = PromptTemplate(
    template='Write a joke about {topic}',
    input_variables=['topic'])


parser = StrOutputParser()

prompt2 = PromptTemplate(
    template='Explain the following joke - {text}',
    input_variables=['text']
)

chain = prompt1 | model | parser | prompt2 | model | parser

print(chain.invoke({'topic':'AI in DevOps'}))

This joke is funny because it combines our understanding of typical developer workflows with the reference to AI's ability, even in its simulated state, to have straightforward goals. 

Here's the breakdown:

* **"Unit tests passed, now just need to run this script..."** -  This is a familiar line for programmers, it signifies they're close to deploying their code. It plays on the expectation of a simple script for deployment. 
* **"Endless checklist"** - This refers to the complex and extensive list of processes and steps that are usually involved with deploying any software, even to AI systems. This checklist is often daunting and can seem endless, especially when dealing with potentially sensitive nature of how AI is deployed. 
* **"Developer goggles"** - This is a humorous reference to how developers sometimes wear specialized mode which helps them focus on details. 

**Putting it together:**

The joke satirizes the stressful, intricate, and slightly comical aspects of deploying an



---



# **Runnable Parallel**

In [14]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.runnables import RunnableParallel



prompt1 = PromptTemplate(
    template='Generate a tweet about {topic}',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Generate a Linkedin post about {topic}',
    input_variables=['topic']
)

llm = HuggingFaceEndpoint(repo_id='google/gemma-2-2b-it', task='text-generation', huggingfacehub_api_token='your api key')

model = ChatHuggingFace(llm=llm)

parser = StrOutputParser()

tweet_chain    = prompt1 | model | parser
linkedin_chain = prompt2 | model | parser

parallel_chain = RunnableParallel({
    'tweet': tweet_chain,
    'linkedin': linkedin_chain})


result = parallel_chain.invoke({'topic':'Fitness'})

print(result['tweet'])
print(result['linkedin'])


Here are a few tweet options about fitness, varying in tone and focus:

**Motivational:**

* Breaking out my workout clothes and getting ready to crush it! 💪 Let's gooooooo! #fitness #motivation #healthylifestyle
* Feeling strong and energized after an amazing workout. ⚡️ Nothing beats the feeling of accomplishment! #proudofmyself #fitnessjourney

**Informative/Sharing an Achievement:**

* Finally hit my personal best for squats today! 🎉 Feeling so satisfied! #fitnessgains #gains #exercise 
* Learning something new about fitness today - trying out foam rolling for the first time! Feeling the difference already. 🤸‍♀️ 

**Humorous:**

* My legs feel like jelly, but my feeling is solid. I'll take victory over exhaustion any day. 😂 #fitnessfail #MondayMotivation

**General:**

* Active lifestyles are the best way to live. What is your favorite way to stay active? #wellness #healthyliving 
* Feeling inspired to trade my couch for miles today. Do you have your workout clothes ready?  🏃‍♀️ #g

- You define two prompt templates (one for a tweet, one for a LinkedIn post), connect each prompt to the same LLM and string parser, then run both chains in parallel with the same input topic ("Fitness"). The result is a dictionary with two generated texts: one for the tweet and one for the LinkedIn post.

- RunnableParallel executes multiple runnables concurrently,
When the RunnableParallel is invoked the same input is passed to each child runnable and Each child runnable returns a result.

- parallel_chain.invoke({'topic':'Fitness'}) runs the pipeline synchronously and returns the aggregated outputs.
Both tweet_chain and linkedin_chain receive {"topic": "Fitness"}, build their respective prompts, call the LLM, parse the outputs, and return them inside result.




---





---



# **Runnable Passthrough & Runnable Lambda**

In [16]:
# LangChain runnable primitives
from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough,
    RunnableLambda,
)
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Hugging Face endpoint + chat wrapper
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace


# create HF endpoint LLM (adjust repo_id & task if needed)
llm_endpoint = HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it",
    task="text-generation",
    huggingfacehub_api_token=("your api key"),
)

# chat wrapper around the endpoint
model = ChatHuggingFace(llm=llm_endpoint)

# prompts and parser
prompt1 = PromptTemplate(
    template="Write a joke about {topic}",
    input_variables=["topic"],
)

prompt2 = PromptTemplate(
    template="Explain the following joke - {text}",
    input_variables=["text"],
)

parser = StrOutputParser()

# 1) joke generation chain: prompt1 -> model -> parser (produces a plain string)
joke_gen_chain = prompt1 | model | parser

# 2) small mapper runnable: string joke -> {"text": joke}
to_text_dict = RunnableLambda(lambda joke: {"text": joke})

# 3) explanation chain: prompt2 -> model -> parser (expects {"text": ...})
explain_chain = prompt2 | model | parser

# 4) parallel: keep joke as-is, and run explanation by first mapping to {"text": joke}
parallel_chain = RunnableParallel(
    {
        "joke": RunnablePassthrough(),               # returns the raw joke string
        "explanation": to_text_dict | explain_chain, # converts joke -> {"text": joke} then explains
    }
)

# 5) final pipeline: generate joke, then run the parallel stage (so both values returned)
final_chain = joke_gen_chain | parallel_chain

# invoke with the single input required by the first stage
result = final_chain.invoke({"topic": "cricket"})
print(result)  # -> {"joke": "....", "explanation": "...."}


{'joke': "Why did the cricket blush? \n\nHe saw the ball coming in at the crease! 😆 🏏 \n\n\nLet me know if you'd like to hear another! 🏏 😊 \n", 'explanation': 'This is a classic dad-joke-style silly play on words! Here\'s the breakdown:\n\n* **The setup:**  The joke relies on our expectation of a typical cricket joke about a cricket\'s action. \n* **The punchline:**  The key lies in the word "crease." In cricket, the crease is where the batsman stands to face the pitch where the ball is bowled.\n* **The pun:** "Crease" sounds like a synonym for "blush," making a funny connection between the physical action of blushing due to seeing something (a ball) and a context (the crease meant as a baseball metaphor).\n\n😂  It\'s a light-hearted play on words, humorous because it\'s unexpected and creates a ridiculous juxtaposition. \n\n\nWould you like another cricket joke? 🤔  I\'ve got a few more up my sleeve! 😊🏏 \n'}


- joke_gen_chain = prompt1 | model | parser <br>
prompt1 formats the prompt, <br>
model generates text,<br>
parser converts the result to str. <br>
- RunnableLambda(lambda joke: {"text": joke}) wraps a Python lambda as a Runnable and converts a raw joke string into the dictionary {"text": joke} which matches prompt2's expected input variable name.
- explain_chain = prompt2 | model | parser <br> is similar to joke_gen_chain, but it expects an input dict containing text (the joke).

- parallel_chain = RunnableParallel() takes a mapping of keys to runnables and runs them "in parallel" over the same input.

  - RunnablePassthrough() returns that same joke string

  - The 'explanation' pipeline first transforms the string into {"text": joke} and then runs the explanation chain which produces the explanation string.

  - The output from parallel_chain is a dictionary: {"joke": <string>, "explanation": <string>}.

- final_chain = joke_gen_chain | parallel_chain <br>
joke_gen_chain takes {"topic": "cricket"}  returns the joke string. <br>
That joke string becomes the input to parallel_chain. <br>
parallel_chain returns {"joke": joke_string, "explanation": explanation_string}. <br>
So final_chain.invoke({"topic":"cricket"}) returns the final dict in one call.



# FLOW

1. Caller: {"topic": "cricket"} (dict)

2. joke_gen_chain:

   - Input: dict {"topic": "cricket"}

   - Output: joke_string (type str)

3. parallel_chain receives joke_string and runs both children:

   - RunnablePassthrough()  returns joke_string

   - to_text_dict (RunnableLambda) receives joke_string returns {"text": joke_string}

   - explain_chain receives {"text": joke_string} returns explanation_string (type str)

4. parallel_chain aggregates outputs {"joke": joke_string, "explanation": explanation_string}



---





---



# **Runnable Branch**

In [18]:
# modern runnable primitives
from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough,
    RunnableBranch,
    RunnableLambda,
)
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Hugging Face endpoint + chat wrapper
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace



# create HF endpoint LLM (adjust repo_id & task if you want another model)
llm_endpoint = HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it",    # change if you prefer another HF model
    task="text-generation",
    huggingfacehub_api_token=("your api key"))

model = ChatHuggingFace(llm=llm_endpoint)

# prompts and parser
prompt1 = PromptTemplate(
    template="Write a detailed report on {topic}",
    input_variables=["topic"],
)

prompt2 = PromptTemplate(
    template="Summarize the following text\n\n{text}",
    input_variables=["text"],
)

parser = StrOutputParser()

# 1) report generation chain: prompt1 -> model -> parser  (produces string)
report_gen_chain = prompt1 | model | parser

# 2) summary chain: prompt2 -> model -> parser (expects {"text": ...})
summary_chain = prompt2 | model | parser

# 3) branch: if report is long (>300 words) -> run summary_chain, else passthrough (return report unchanged)
#    Note: condition receives the *report string* from report_gen_chain

branch_chain = RunnableBranch(
    (lambda x: len(x.split()) > 300, summary_chain),  # if True -> summarize
    RunnablePassthrough(),                             # else -> return original report
)

# 4) parallel: produce both 'report' and 'summary' keys for final output
parallel_chain = RunnableParallel(
    {
        "report": RunnablePassthrough(),   # returns the raw report string
        "summary": branch_chain,           # either a summary string or the original report
    }
)

# 5) final pipeline: generate report, then run the parallel stage to return both values
final_chain = report_gen_chain | parallel_chain

# invoke with the input required by the first stage
result = final_chain.invoke({"topic": "Russia vs Ukraine"})

# result is a dict: {"report": "<full report text>", "summary": "<summary or original report>"}
print("=== Report ===\n", result["report"][:1800], "...")   # print first 1000 chars to keep colab tidy
print("\n === Summary / Fallback === \n", result["summary"][:1800], "...")


=== Report ===
 ##  Russia-Ukraine Conflict: A Detailed Report

The ongoing conflict between Russia and Ukraine is a multifaceted crisis with profound geopolitical, humanitarian, and socio-economic consequences. This report provides a detailed examination of various aspects of this rapidly evolving situation:

**I. Historical Context and Background:**

* **Shared History:** Russia and Ukraine share centuries of intertwined history, with both nations tracing their roots back to the Kyivan Rus' civilization. This shared history has shaped cultural ties and ideological perceptions. 
* **Soviet Era:**  Ukraine was part of the Soviet Union until 1991, when it declared its independence. Following this, relations between the two nations were characterized by complex interactions, including economic dependence, political tensions, and cultural differences.
* **Post-Soviet Period:** Despite attempts at democratic transition, Ukraine sought closer ties with the West while facing challenges from 

- prompt1 will create a prompt like "Write a detailed report on Russia vs Ukraine" when given {"topic": "Russia vs Ukraine"}.
- prompt2 expects a text variable (the report)

- report_gen_chain = prompt1 | model | parser
Formats the prompt from {"topic": ' '}  a string prompt, <br>
Calls the LLM to generate a report, <br>
Parses to a string. <br>
Input type: {"topic": str} Output type: str (the full report).

- summary_chain = prompt2 | model | parser <br>
expects an input dict {"text": <report string>} and returns a summary str.

- branch_chain = RunnableBranch() <br>
RunnableBranch takes branching rules. <br>

  - A tuple (predicate, runnable_if_true): predicate is lambda x: len(x.split()) > 300 <br>
  x.split() splits on whitespace, len(...) > 300 checks if the report has more than 300 words <br>
  If True summary_chain is executed.

  - A second argument is the runnable to run if predicate is false: RunnablePassthrough().

- parallel_chain = RunnableParallel() <br>
runs both children with the same input <br>
"report" uses RunnablePassthrough() so that the final output includes the full report unchanged. <br>
"summary" runs the branch. The branch returns either:
a summary string (if report > 300 words), or <br>
the original report (via passthrough) if it is short.

- final_chain = report_gen_chain | parallel_chain <br>
full flow: <br>
report_gen_chain (input: {"topic": ...} -> output: report_str) <br>
parallel_chain (input: report_str -> output: {"report": report_str, "summary": summary_or_report}) <br>
Final invoke returns the dict described.

# Flow

1. Caller passes {"topic": str} to final_chain. <br>
2. report_gen_chain returns report_str (type str) <br>
3. parallel_chain gets report_str and runs: <br>
 "report" child returns report_str (passthrough). <br>
 "summary" child → runs branch_chain:
   - If True: run summary_chain (which expects {"text": report_str}) and returns summary string. <br>
   - If False: RunnablePassthrough() returns report_str. <br>
4. parallel_chain returns {"report": report_str, "summary": ...}



---





---



1. PromptTemplate for structured prompt construction

2. HuggingFaceEndpoint + ChatHuggingFace to run LLM models

3. StrOutputParser to clean and normalize model output

4. RunnableSequence (| operator) to chain steps (prompt -> model -> parser)

5. RunnableParallel for multi-branch outputs (tweet + LinkedIn, report + summary)

6. RunnableLambda for lightweight transformations (string -> dict mappings)

7. RunnableBranch for conditional logic (summarize only if long)

8. RunnablePassthrough for retaining upstream outputs without modification



---



1. Ensure Input–Output Shape Consistency <br>
Always confirm that downstream runnables (eg summarizers) receive the structured input they expect. Use RunnableLambda for safe transformations.

2. Prefer JSON-Structured Outputs <br>
When possible, request JSON and parse with structured output parsers for reliability and downstream automation.

3. Use ainvoke() + asyncio when calling multiple models or long-generation pipelines to reduce wall-time.

4. Secure API Keys  <br>
Use environment variables or secret managers—avoid hardcoding tokens in pipeline code.